In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from cognite.client.data_classes.contextualization import VisionFeature
from connect import init_cognite_connect

creds, config, client = init_cognite_connect()

### Find relevant images:

In [2]:
## sigrid-data
sigrid_site_id = "site-6823646490256602-4885115113942810"
sigrid_image_list = client.files.list(limit=-1, metadata={"site_id":sigrid_site_id})
print(len(sigrid_image_list)) #should be 48 as of 18.04

48


In [3]:
file_ids_sigrid = []
for file in sigrid_image_list:
    file_ids_sigrid.append(file.id)

137673760555654

In [4]:
extract_job_sigrid = client.vision.extract(
    features=VisionFeature.ASSET_TAG_DETECTION,file_ids=file_ids_sigrid)

In [5]:
extract_job_sigrid.wait_for_completion()

In [6]:
extract_job_sigrid.result

{'items': [{'fileExternalId': 'image-station-6823646490256602-4885115113942810-200544-MPS Dense Area 1 1-334-0000000000000001-0',
   'fileId': 7977850905805347,
   'predictions': {'assetTagPredictions': []}},
  {'fileExternalId': 'image-station-6823646490256602-4885115113942810-200544-MPS Dense Area 1 1-331-0000000000000001-2',
   'fileId': 8128331645382016,
   'predictions': {'assetTagPredictions': []}},
  {'fileExternalId': 'image-station-6823646490256602-4885115113942810-200544-MPS Dense Area 1 1-337-0000000000000001-2',
   'fileId': 8447895193985214,
   'predictions': {'assetTagPredictions': []}},
  {'fileExternalId': 'image-station-6823646490256602-4885115113942810-200544-MPS Dense Area 1 1-332-0000000000000001-5',
   'fileId': 8596438045071458,
   'predictions': {'assetTagPredictions': []}},
  {'fileExternalId': 'image-station-6823646490256602-4885115113942810-200544-MPS Dense Area 1 1-331-0000000000000001-4',
   'fileId': 8665276623490316,
   'predictions': {'assetTagPredictions

### Trying with API:

In [15]:
import os
import requests

# Your variables
TOKEN_URL = os.getenv("TOKEN_URL")
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
CDF_CLUSTER = os.getenv("CDF_CLUSTER")
BASE_URL = f"https://{CDF_CLUSTER}.cognitedata.com"
SCOPES = [f"{BASE_URL}/.default"]
COGNITE_PROJECT = os.getenv("COGNITE_PROJECT")

#OAuth token:
token_data = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials",
    "scope": " ".join(SCOPES),
}
token_r = requests.post(TOKEN_URL, data=token_data)
token = token_r.json().get("access_token")

# API request
url = f"https://{CDF_CLUSTER}.cognitedata.com/api/v1/projects/{COGNITE_PROJECT}/context/vision/extract"
headers = {
    "Authorization": f"Bearer {token}",
    "accept": "application/json",
    "content-type": "application/json",
    "cdf-version": "beta"  # Or try 20230101-beta
}

# Payload 
## Points for payload
file_id_test = 2530661728975954
payload = {
    "items": [{"fileId": file_id_test}], 
    "features": ["AssetTagDetection"],
}

# Make the API request
response = requests.post(url, json=payload, headers=headers)

# Handle response
if response.status_code == 200:
    data = response.json()
    # Your code to handle the data here
    print("Data received:", data)
else:
    print("Error:", response.status_code, response.text)


def check_job_status(job_id):
    # Endpoint to check the status of the segmentation job
    status_url = f"https://{CDF_CLUSTER}.cognitedata.com/api/v1/projects/{COGNITE_PROJECT}/context/vision/extract/{job_id}"

    # Headers with "beta" version
    headers = {
        "Authorization": f"Bearer {token}",
        "accept": "application/json",
        "content-type": "application/json",
        "cdf-version": "beta"
    }

    # Make the API request
    response = requests.get(status_url, headers=headers)

    # Handle response
    if response.status_code == 200:
        data = response.json()
        job_status = data.get("status")
        if job_status == "Completed":
            print("The segmentation job is completed!")
            return data
        elif job_status == "Queued" or job_status == "Running":
            print(f"The segmentation job is currently {job_status}. Please wait...")
            return False
        else:
            print(f"Unknown job status: {job_status}")
            print(data)
            return False
    else:
        print("Error:", response.status_code, response.text)
        return False

Data received: {'createdTime': 1713448229671, 'features': ['AssetTagDetection'], 'items': [{'fileId': 2530661728975954}], 'jobId': 6718733810130210, 'status': 'Queued', 'statusTime': 1713448229671}


In [16]:
check_job_status(6699477637134437)

Unknown job status: Failed
{'createdTime': 1713447836569, 'failedItems': [{'errorMessage': "JobFailedException: TypeError('CountAggregate' object is not subscriptable)", 'items': [{'fileId': 2530661728975954}]}], 'jobId': 6699477637134437, 'startTime': 1713447839294, 'status': 'Failed', 'statusTime': 1713448047541}


False

In [17]:
check_job_status(6718733810130210)

Unknown job status: Failed
{'createdTime': 1713448229671, 'failedItems': [{'errorMessage': "JobFailedException: TypeError('CountAggregate' object is not subscriptable)", 'items': [{'fileId': 2530661728975954}]}], 'jobId': 6718733810130210, 'startTime': 1713448231062, 'status': 'Failed', 'statusTime': 1713448233746}


False

## Continue with results

In [31]:
for pred in extract_job_sigrid.items:
    fileID = pred.file_id
    assettagpreds = pred.predictions.asset_tag_predictions
    if assettagpreds:
        print("FILE ID:",fileID)
        print(assettagpreds)

FILE ID: 3412097324633202
[AssetLink(text='IL-42-A-9A9', text_region=BoundingBox(x_min=0.505859375, x_max=0.5384765625, y_min=0.5453125, y_max=0.552734375), asset_ref=CdfResourceRef(id=2194542392124481, external_id=None), confidence=1.0)]
FILE ID: 6019921028862041
[AssetLink(text='BE-5526A', text_region=BoundingBox(x_min=0.27187500000000003, x_max=0.2951171875, y_min=0.4578125, y_max=0.46503906250000004), asset_ref=CdfResourceRef(id=7636371139593460, external_id=None), confidence=1.0)]


In [32]:
extract_job_sigrid.save_predictions()

,annotation_type,data,status,creating_app,creating_app_version,annotated_resource_type,annotated_resource_id,id,created_time,last_updated_time,creating_user
0,images.AssetLink,"{'assetRef': {'id': 2194542392124481}, 'confid...",suggested,cognite-sdk-python,7.37.1,file,3412097324633202,4865406478955850,2024-04-22 07:16:43.146,2024-04-22 07:16:43.146,None
1,images.AssetLink,"{'assetRef': {'id': 7636371139593460}, 'confid...",suggested,cognite-sdk-python,7.37.1,file,6019921028862041,4155185497432281,2024-04-22 07:16:43.147,2024-04-22 07:16:43.147,None
